In [1]:
import numpy as np
from tensorflow.keras import layers, models

# Define the CNN model
def cnn_model(input_shape):
    """

    Args:
      input_shape:

    Returns:

    """
    model = models.Sequential()

    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Flatten the output for the dense layers
    model.add(layers.Flatten())

    # Fully connected layers
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))

    # Output layer (5 classes: W, S, A, D, N)
    model.add(layers.Dense(5, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


In [2]:
train_data = np.load('training_data_v7.2.npy', allow_pickle=True)

In [3]:
# Prepare the data
X = np.array([i[0] for i in train_data]).reshape(-1, 80, 60, 1)  # 80x60 grayscale images
Y = np.array([i[1] for i in train_data])  # Labels (one-hot encoded)

# Split into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [4]:
# Initialize the model
input_shape = (80, 60, 1)  # 80x60 grayscale images
model = cnn_model(input_shape)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 78, 58, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 39, 29, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 37, 27, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 18, 13, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 16, 11, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 8, 5, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 5120)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       1,310,976 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,437,189 (5.48 MB)

 Trainable params: 1,437,189 (5.48 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Train the model
model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_data=(X_test, Y_test))

# Save the model
model.save('talking_tom_cnn_model_100_epochs.h5')

Epoch 1/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 11s 297ms/step - accuracy: 0.2374 - loss: 13.5087 - val_accuracy: 0.3468 - val_loss: 1.4427
Epoch 2/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 9s 245ms/step - accuracy: 0.3091 - loss: 1.4584 - val_accuracy: 0.4355 - val_loss: 1.3414
Epoch 3/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 9s 218ms/step - accuracy: 0.4007 - loss: 1.3667 - val_accuracy: 0.5565 - val_loss: 1.1341
Epoch 4/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 10s 217ms/step - accuracy: 0.4826 - loss: 1.1581 - val_accuracy: 0.5565 - val_loss: 1.0215
Epoch 5/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 12s 260ms/step - accuracy: 0.6144 - loss: 0.9609 - val_accuracy: 0.6331 - val_loss: 0.8434
Epoch 6/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 10s 269ms/step - accuracy: 0.6471 - loss: 0.8729 - val_accuracy: 0.7460 - val_loss: 0.7495
Epoch 7/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 9s 218ms/step - accuracy: 0.6996 - loss: 0.7456 - val_accuracy: 0.7500 - val_loss: 0.5869
Epoch 8/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 11s 241ms/step - accuracy: 0.7894 - loss: 0.5734 - va

In [7]:
model.evaluate(X_test, Y_test)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.9685 - loss: 0.2236


[0.24072083830833435, 0.9637096524238586]

In [9]:
from tensorflow.keras.models import load_model

# Load the model
model = load_model('talking_tom_cnn_model_100_epochs.h5')

# Print the model summary (optional)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 78, 58, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 39, 29, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 37, 27, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 18, 13, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 16, 11, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 8, 5, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 5120)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       1,310,976 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,437,191 (5.48 MB)

 Trainable params: 1,437,189 (5.48 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [ ]:
import os
import numpy as np
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint

def cnn_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Specify model path
MODEL_PATH = 'models/talking_tom_cnn_model_1.h5'
DATA_FOLDER = 'data/training_batches'  # Folder containing training data
MODEL_SAVE_INTERVAL = 5  # Save model every 5 epochs
BATCH_SIZE = 32
EPOCHS = 10

# Load pre-trained model or create a new one
input_shape = (80, 60, 1)
if os.path.exists(MODEL_PATH):
    print(f"Loading pre-trained model from {MODEL_PATH}")
    try:
        model = load_model(MODEL_PATH)
    except Exception as e:
        print(f"Error loading model: {e}")
        print("Creating a new model.")
        model = cnn_model(input_shape)
else:
    print("No pre-trained model found. Creating a new model.")
    model = cnn_model(input_shape)

# Initialize ModelCheckpoint to save model periodically
checkpoint_callback = ModelCheckpoint(
    MODEL_PATH,
    save_best_only=True,
    save_weights_only=False,
    verbose=1,
)

# Iterate through training data files
data_files = [f for f in os.listdir(DATA_FOLDER) if f.endswith('.npy')]
if not data_files:
    raise FileNotFoundError("No training data files found in the specified folder.")

for file_idx, file_name in enumerate(data_files):
    try:
        print(f"Loading data from file: {file_name}")
        file_path = os.path.join(DATA_FOLDER, file_name)
        train_data = np.load(file_path, allow_pickle=True)

        # Prepare data
        X = np.array([i[0] for i in train_data]).reshape(-1, 80, 60, 1)
        Y = np.array([i[1] for i in train_data])  # Labels (one-hot encoded)

        # Split into training and testing sets
        X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

        print(f"Training on data file {file_idx + 1}/{len(data_files)}: {file_name}")

        # Train model
        model.fit(
            X_train, Y_train,
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            validation_data=(X_val, Y_val),
            callbacks=[checkpoint_callback]
        )

    except Exception as e:
        print(f"An error occurred while processing {file_name}: {e}")
        continue

# Save final model state
try:
    print(f"Saving final model to {MODEL_PATH}")
    model.save(MODEL_PATH)
except Exception as e:
    print(f"Error saving final model: {e}")

print("Training complete.")
